In [1]:
!pip install -U transformers
!python -m pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 56.0 MB/s eta 0:00:00


In [2]:
import av
import torch
import json
import os
import numpy as np
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [4]:
# Load the model

model = VideoLlavaForConditionalGeneration.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/66.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
def infer(queries_file, video_dir, output_file, num_new_tokens, prompt_style):
  predictions = []


  # Load the prompts
  with open(queries_file, 'r') as quer_file:
      queries = json.load(quer_file)


  for query in queries:
      prompt = query["query"]
      video_id = query["video_id"]
      ground_truth = query["ground_truth"]
      start_time, end_time = ground_truth

      video_path = os.path.join(video_dir, f"{video_id}__{start_time}__{end_time}.mp4")

      if not os.path.exists(video_path):
          print(f"Video file not found: {video_path}")
          continue

      try:
          container = av.open(video_path)
          if not container.streams.video:
              print(f"No video stream found in file: {video_path}")
              continue

          total_frames = container.streams.video[0].frames
          if total_frames == 0:
              print(f"No frames in video: {video_path}")
              continue

          indices = np.arange(0, total_frames, total_frames / 8).astype(int)
          video = read_video_pyav(container, indices)
      except Exception as e:
          print(f"Error processing video {video_path}: {e}")
          continue


      if prompt_style == "user_assistant":
        # For better results, we recommend to prompt the model in the following format
        prompt = f"USER: <video>\n{prompt} ASSISTANT:"
      elif prompt_style == "raw":
        prompt = prompt

      inputs = processor(text=prompt, videos=video, return_tensors="pt")
      inputs = {key: value.to(device) for key, value in inputs.items()}


      out = model.generate(**inputs, max_new_tokens=num_new_tokens)
      full_output = processor.batch_decode(
          out, skip_special_tokens=True, clean_up_tokenization_spaces=True
      )[0]

      # Extract response (after "ASSISTANT:")
      response = full_output.split("ASSISTANT:")[-1].strip()

      # Save the result in the predictions list
      predictions.append({
          "video_id": video_id,
          "prompt": prompt,
          "prediction": response,
      })

      print(f"Processed video {video_id}")

  # Save predictions to a JSON file
  with open(output_file, 'w') as f:
      json.dump(predictions, f, indent=4)

  print(f"Predictions saved to {output_file}")



In [6]:
queries_file = "/content/drive/MyDrive/data/top_50_queries/top_queries.json"
video_dir = "/content/drive/MyDrive/data/extracted_clips"

In [ ]:
# We now run VideoLLava with different configurations

# First is max 20 tokens and suggested prompt style (user and assitant placeholders)

output_file = "/content/drive/MyDrive/data/videollava_20tokens_user_assistant_prompt.json"  # Output file for predictions
infer(queries_file=queries_file, video_dir=video_dir, output_file=output_file, num_new_tokens=20, prompt_style="user_assistant")

# Second is max 20 tokens and raw prompt style

output_file = "/content/drive/MyDrive/data/videollava_20tokens_raw_prompt.json"  # Output file for predictions
infer(queries_file=queries_file, video_dir=video_dir, output_file=output_file, num_new_tokens=20, prompt_style="raw")

# Third is max 1024 tokens and suggested prompt style (user and assitant placeholders)

output_file = "/content/drive/MyDrive/data/videollava_1024tokens_user_assistant_prompt.json"  # Output file for predictions
infer(queries_file=queries_file, video_dir=video_dir, output_file=output_file, num_new_tokens=1024, prompt_style="user_assistant")

# Fourth is max 1024 tokens and raw prompt style


output_file = "/content/drive/MyDrive/data/videollava_1024tokens_raw_prompt.json"  # Output file for predictions
infer(queries_file=queries_file, video_dir=video_dir, output_file=output_file, num_new_tokens=1024, prompt_style="raw")





Processed video 8698a0ea-f434-49f9-a8e4-45220e5d4b2c
Processed video f082242c-309f-48a1-97fa-5c1d6bd255fb
Processed video 0971527a-6cd3-4c82-9d94-09b3565f4505
Processed video 4ce119de-0f42-4bd1-b387-9e19643fdddc
